<a href="https://colab.research.google.com/github/Fiona-cherutto/deployone/blob/main/Copy_of_Copy_of_NLP_test_model_CRM(with_location)_successes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import xlrd
import nltk
!pip install nltk==3.5
!pip install regex==2020.6.8
!pip install joblib==0.16.0
!pip install tqdm==4.47.0
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df =  pd.read_excel('/content/1000 leads.xlsx')


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [4]:
df = df.rename(columns={'Status information':'Status_information','Status ':'Status'})

In [5]:
df["Location"] = df["Location"].str.lower()

In [6]:
pattern = r"\d*\/\d*\/[a-zA-z]*:|\d*\/\d*\/\d*\(\w*\):|\d*\|\w*\|\w*:|\(\w*\):|\d*\-\d*\-\d*\(\w*\):|\d*\/\d*\w*:|\d*\/\w*:|\d*\/\d*\/\w*\|:|\d*\/\d*\ *\(\w*\)|\d*\ *\w*\ *\|\ *\w*\ *:|\d*\/\d*\/\d*\(\w*:|\d*\/\w*\/\w*|\d*\/\d*\/\d*\|\(\w*\)|\(\w*\)|\d*\-\d*\-\d*|^\ *|^\:"

In [7]:
df["Status_information"] = df["Status_information"].apply(lambda x: re.sub(pattern," ",str(x)))

In [8]:
del df['Unnamed: 4']
del df["Lead Name"]
del df["Location"]

In [9]:
df.dropna(inplace=True)

In [10]:
df['Status'] = df['Status'].str.lower()
df['Status'] = df["Status"].replace(' ','')

df['Status'] = df["Status"].str.replace('conveted','converted')

df['Status'] = df["Status"].str.replace('converted ','converted')

In [11]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df[df['Status'] == 'not converted']
df_minority = df[df['Status'] == 'converted']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=123) # random
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [12]:
def message_text_process(mess):
  no_punctuation = [char for char in mess if char]
  no_punctuation = "".join(no_punctuation)
  return [word for word in no_punctuation.split() if word.lower()
          not in stopwords.words("english")]

In [13]:
df_upsampled['Status_information'] = df_upsampled['Status_information'].apply(message_text_process).astype(str)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer #bag of words
#data = message_text_process(df_upsampled["Status_information"])
#count_vec = CountVectorizer(analyzer=message_text_process)
count_vec = CountVectorizer()
X = count_vec.fit_transform(df_upsampled["Status_information"])
#X = count_vec.fit_transform(df_upsampled["Status_information"])

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer #TF.IDF
tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)

In [16]:
X.shape

(1734, 1245)

In [17]:
X

<1734x1245 sparse matrix of type '<class 'numpy.float64'>'
	with 14330 stored elements in Compressed Sparse Row format>

In [18]:
X.shape

(1734, 1245)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df_upsampled['Status'], test_size=0.2, random_state=42)

In [20]:
#random foresset classifier with TFIDF
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
Random_Forest_Classifier = RandomForestClassifier()
forest = Random_Forest_Classifier.fit(X_train,y_train)
predicted = forest.predict(X_test)
expected = y_test
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

               precision    recall  f1-score   support

    converted       0.95      0.98      0.96       164
not converted       0.98      0.95      0.96       183

     accuracy                           0.96       347
    macro avg       0.96      0.96      0.96       347
 weighted avg       0.96      0.96      0.96       347

[[160   4]
 [  9 174]]


In [21]:
import pickle

In [22]:
pickle_out = open('Random_Forest_Classifier.pkl', 'wb')
pickle.dump(Random_Forest_Classifier, pickle_out)
pickle_out.close()

In [23]:
pickle_out = open('tfidf_Transformer.pkl', 'wb')
pickle.dump(tfidf, pickle_out)
pickle_out.close()

In [24]:
pickle_out = open('count_vec.pkl', 'wb')
pickle.dump(count_vec, pickle_out)
pickle_out.close()

In [25]:
def test_model():
  test_text=input()
  location_label = np.array(29).reshape(-1,1)

  
  bag_words_test = message_text_process(test_text)
  # bag_words =  CountVectorizer(analyzer=message_text_process).fit(test_tex])
  message_bagwords_test = count_vec.transform([test_text])
  #tfid_transformer = TfidfTransformer().fit(message_bagwords)
  message_tfidf = tfidf.transform(message_bagwords_test)
  #message_tfidf = csr_matrix.toarray(message_tfidf)
  #data = np.append(message_tfidf,location_label,axis=1)
  
  y_pred = Random_Forest_Classifier.predict(message_tfidf)

  print(y_pred[0])
  # print(location_label.shape)


In [27]:
test_model()

 i am not interested
not converted


In [28]:
variable = "hello i am interested"

# variable = np.array()
# variable = np.append(variable)

x = pd.DataFrame(variable)

ValueError: ignored

In [ ]:
x.values[0][0]

'hello i am interested'

In [ ]:
pip install streamlit

In [1]:
import streamlit as st
import pickle
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')





BOW = pickle.load(open('count_vec.pkl', 'rb'))
RF=pickle.load(open('Random_Forest_Classifier.pkl','rb'))
tfidf=pickle.load(open('tfidf_Transformer.pkl','rb'))


#tockenization and removing punctutations
def message_text_process(data):
  no_punctuation = [char for char in data if char]
  no_punctuation = "".join(no_punctuation)
  return [word for word in no_punctuation.split() if word.lower()
          not in stopwords.words("english")]



#data cleaning
def string_cleaning(data):
    
    pattern = r"\d*\/\d*\/[a-zA-z]*:|\d*\/\d*\/\d*\(\w*\):|\d*\|\w*\|\w*:|\(\w*\):|\d*\-\d*\-\d*\(\w*\):|\d*\/\d*\w*:|\d*\/\w*:|\d*\/\d*\/\w*\|:|\d*\/\d*\ *\(\w*\)|\d*\ *\w*\ *\|\ *\w*\ *:|\d*\/\d*\/\d*\(\w*:|\d*\/\w*\/\w*|\d*\/\d*\/\d*\|\(\w*\)|\(\w*\)|\d*\-\d*\-\d*|^\ *|^\:"        
    
    data = re.sub(pattern," ",data)
    
    return data





#Random Forest Classifier
def predict_(data):
    
    prediction = RF.predict(data)
    
    return prediction
    
    





def main():
    
    
        st.title("Customer Relationship Management")
        
        # phone_number = st.text_input('Customer Phone number with country code!(which is also Unique Id)')
    
        # cities = {'hyderabad':16,'pune':35,'mumbai':27, 'delhi':8,'bangalore':3,'australia':2,
        #        'nagpur':29,'madurai':25,'mysore':28,'kerala':21,'chennai':6,'jalandhar':19,
        #        'tiruttani':40,'usa':42,'faridabad':9,'online':34,'gurgoan':14,'kochi':23,
        #        'noida':32,'ahmedabad':0,'khammam':22,'vishakapatnam':44,'solapur':38,
        #        'nasik':30,'thane':39,'uae':41,'aurangabad':1,'rayagada':37,'bilgi':5,
        #        'gurgaon':13,'rajamundry':36,'kadapa':20,'vijayawada':43,'kolkatta':24,
        #        'meerut':26,'jaipur':18,'coimbatore':7,'gujarat':11,'ghazibad':10,'india':17,
        #        'ongole':33,'nepal':31,'guntur':12,'hubli':15,'bihar':4}
        
        
        
        # Location = st.selectbox("Select a Location from the drop Down meanu", list(cities.keys()))
        
        
        
        
        if st.button("input text box"):
            
            message = st.text_input("Customer Response")
            
            data =[message]
            
            data = string_cleaning(data)
            
            data = data.apply(message_text_process).astype(str)       
            
            data = BOW.transform(data)
        
            data = tfidf.transform(data)
            
            
        

        
        if st.button("Predict"):
            
            result=predict_(data)
            
            if result == ['not converted']:
                st.success("This lead will not going get converted probabily")
                
            else:
                
                st.successs("This lead will probabily going to get converted! Wish you luck!")
            
            
        if st.button("About"):
            st.text("Lets Learn")
            st.text("Built with Streamlit")
            
    
            
            
            
            
            
            
            


if __name__=='__main__':
        
            
        main()
        
        #message_text_process(data)
        

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2021-02-11 14:01:14.169 WARNING root: 
  command:

    streamlit run /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py [ARGUMENTS]
